### Load the packages

Code from Chin-Chieh Wu (Chang Gung University)

In [ ]:
# Install the renv package if not already installed
if (!requireNamespace("renv", quietly = TRUE)) {
  install.packages("renv")
}

# Load or create a project-specific environment
if (!file.exists("renv.lock")) {
  renv::init()
} else {
  renv::activate()
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



The following package(s) will be updated in the lockfile:

# CRAN -----------------------------------------------------------------------
- renv   [* -> 1.0.0]

The version of R recorded in the lockfile will be updated:
- R      [* -> 4.3.1]

- Lockfile written to '/content/renv.lock'.


In [ ]:
# Install required packages
if (!requireNamespace("rstan", quietly = TRUE)) {
  install.packages("rstan")
}
if (!requireNamespace("bayesplot", quietly = TRUE)) {
  install.packages("bayesplot")
}
if (!requireNamespace("ggplot2", quietly = TRUE)) {
  install.packages("ggplot2")
}
if (!requireNamespace("doParallel", quietly = TRUE)) {
  install.packages("doParallel")
}
if (!requireNamespace("devtools", quietly = TRUE)) {
  install.packages("devtools")
}

# Downloading packages -------------------------------------------------------

- Downloading rstan from CRAN ...               OK [11.9 Mb in 0.58s]
- Downloading inline from CRAN ...              OK [123.9 Kb in 3.2s]
- Downloading gridExtra from CRAN ...           OK [1.1 Mb in 0.52s]
- Downloading gtable from CRAN ...              OK [210.5 Kb in 0.48s]
- Downloading cli from CRAN ...                 OK [1.2 Mb in 0.81s]
- Downloading glue from CRAN ...                OK [143.5 Kb in 0.79s]
- Downloading lifecycle from CRAN ...           OK [119.8 Kb in 0.81s]
- Downloading rlang from CRAN ...               OK [1.5 Mb in 0.85s]
- Downloading Rcpp from CRAN ...                OK [2 Mb in 0.8s]
- Downloading RcppParallel from CRAN ...        OK [1.8 Mb in 0.5s]
- Downloading loo from CRAN ...                 OK [1.8 Mb in 0.54s]
- Downloading checkmate from CRAN ...           OK [700.7 Kb in 0.5s]
- Downloading backports from CRAN ...           OK [92.8 Kb in 0.75s]
- Downloading mat

In [ ]:
library(rstan)
library(bayesplot)
library(ggplot2)
library(doParallel)
library(devtools)
Sys.setenv(PATH = paste("C:/rtools40/mingw64/bin/", Sys.getenv("PATH"), sep=";"))
Sys.setenv(BINPREF = "C:/rtools40/mingw64/bin/")

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.8, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

This is bayesplot version 1.10.0

- Online documentation and vignettes at mc-stan.org/bayesplot

- bayesplot theme set to bayesplot::theme_default()

   * Does _not_ affect other ggplot2 plots

   * See ?bayesplot_theme_set for details on theme setting

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

Loading required package: usethis



In [ ]:
rstan_options(auto_write=TRUE)
options(mc.cores=parallel::detectCores())
registerDoParallel(4)
getDoParWorkers()

[1] 4

### Define a function to prepare the parameters and data for further Bayesian analysis

In [ ]:
getpara<-function(filename=NULL,names.mk=NULL){
          data.mk<-read.csv(filename)
          names.study<-unique(fixlen.str(data.mk$Study))
          hash.study<-1:length(names.study)
          names(hash.study)<-names.study
          hash.mk<-1:length(names.mk)
          names(hash.mk)<-names.mk
          para<-data.frame(study.names=fixlen.str(data.mk$Study),
          Study=array(hash.study[as.character(fixlen.str(data.mk$Study))]),
                      TP=round(data.mk$Case*data.mk$Sensitivity),
                      TN=round(data.mk$Control*data.mk$Specificity),
                      Dis=data.mk$Case,
                      NDis=data.mk$Control,
                      Test=array(hash.mk[as.character(data.mk$Marker)]))
    return(list(para=para,data=data.mk))
  }


### Define a function to get the upper and lower credible bound (97.5%)

In [ ]:
get.ub.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.975)])
  }

get.lb.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.025)])
  }

### Define a function to get the results from Network Meta-analysis

In [ ]:
get.nma<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	        data.nma<-data.ls$data.nma
	        func.ub<-NULL
        	func.lb<-NULL
	        if(conf==95){
		             func.ub<-get.ub.95
		             func.lb<-get.lb.95
	        } else if(conf==90){
		                  func.ub<-get.ub.90
		                  func.lb<-get.lb.90
	        }
	        data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
          data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
          data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
          data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	        mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                         ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                         lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                         mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                         ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                         lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	        rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                         ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                         lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                         mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                         ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                         lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	        or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                         ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                         lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                         mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                         ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                         lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	        dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                        ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                        lb=array(unlist(lapply(data.nma.dor,func.lb))))
          sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                          ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                          lb=array(unlist(lapply(data.nma.sindex,func.lb))))
          sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                           ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                           lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
          sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                           ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                           lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	        results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                 mu=mu.info,
	                 rr=rr.info,
	                 or=or.info,
	                 dor=dor.info,
                   sindex=sindex.info,
                   sindex_se=sindex_se.info,
                   sindex_sp=sindex_sp.info)
	   return(results.nma)
}

### Define a function to extract results from analysis

In [ ]:
fixlen.str<-function(string.arr=NULL){
                   max.len<-max(nchar(as.character(string.arr)))
	                 rule<-paste('% -',as.character(max.len),'s',sep='')
	                 string.arr.fix<-sprintf(rule,string.arr)
	                 return(string.arr.fix)
}
fit.extract<-function(fit.data=NULL,filename.save=NULL){
	                          results.sum<-summary(fit.data)[[1]]
	                          results.simdata<-fit.data@sim$samples[[1]]
	                          save(results.sum,results.simdata,file=filename.save)
	                          return(results.simdata)
}

##########################################################
###  Network Meta-analysis (stan code)
##########################################################

In [ ]:
stan.code.nma<-'
data{
   int N;  //number of comparison??? - 121
   int Nt; //number of test - 10
   int Ns; //number of study - 72
   int TP[N];
   int Dis[N];  //diseased
   int TN[N];
   int NDis[N]; //non-diseased
   int Study[N];
   int Test[N];
}
parameters{
      matrix[2, Nt] logitmu;
      vector[Ns] nu[2];
      matrix[Ns, Nt] delta[2];
      vector<lower=0>[Nt] tau[2]; //*
      vector<lower=0>[2] sigmab;
      real<lower=-1, upper=1> rho;
}
transformed parameters{
             matrix[Ns, 2] p_i[Nt];
             matrix[2, Nt] MU;
             matrix[2, Nt] RR;
             matrix[2, Nt] OR;
             vector[Nt] DOR;
             vector[Nt] S;
             vector[Nt] S_se;
             vector[Nt] S_sp;
             matrix[Nt, Nt] A;
             matrix[Nt, Nt] B;
             matrix[Nt, Nt] C;
             matrix[Nt, Nt] A_se;
             matrix[Nt, Nt] B_se;
             matrix[Nt, Nt] C_se;
             matrix[Nt, Nt] A_sp;
             matrix[Nt, Nt] B_sp;
             matrix[Nt, Nt] C_sp;
             vector<lower=0>[Nt] tausq[2];
             vector<lower=0>[2] sigmabsq;
             matrix[Nt, Nt] sigmasq[2];
             matrix[Nt, Nt] rhow[2];

    for (i in 1:Ns){
        for (j in 1:2){
            for (k in 1:Nt)
                      p_i[k][i,j] = inv_logit(logitmu[j,k] +  nu[j][i] + delta[j][i,k]);
        }
    }

    for (j in 1:2){
        for (k in 1:Nt){
                  MU[j,k] = mean(col(p_i[k], j));
        }
        tausq[j] = (tau[j]).*(tau[j]);
    }

    for (j in 1:2){
        for (k in 1:Nt){
                  RR[j, k] = MU[j, k]/MU[j, 1];
                  OR[j, k] = (MU[j, k]*(1 - MU[j, 1]))/(MU[j, 1]*(1 - MU[j, k]));
         }
    }

    for (l in 1:Nt){
              DOR[l] = (MU[1, l]*MU[2, l])/((1 - MU[1, l])*(1 - MU[2, l]));

        for(m in 1:Nt){
                 A[l, m] = if_else((MU[1, l] > MU[1, m]) && (MU[2, l] > MU[2, m]), 1, 0);
                 B[l, m] = if_else((MU[1, l] < MU[1, m]) && (MU[2, l] < MU[2, m]), 1, 0);
                 C[l, m] = if_else((MU[1, l] == MU[1, m]) && (MU[2, l] == MU[2, m]), 1, 0);

                 A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
                 B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
                 C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);

                 A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
                 B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
                 C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
        }

        S[l] = (2*sum(row(A, l)) + sum(row(C, l)))/(2*sum(row(B, l)) + sum(row(C, l)));
        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
    }

    sigmabsq = (sigmab).*(sigmab);

    for (j in 1:2){
        for (k in 1:Nt){
            for (l in 1:Nt){
                      sigmasq[j][k,l] = (sigmabsq[j] + tausq[j][k])*((sigmabsq[j] + tausq[j][l]));
                      rhow[j][k,l] = sigmabsq[j]/sqrt(sigmasq[j][k,l]);
            }
        }
    }

}
model{
	   //Priors
       for (j in 1:2){
                 logitmu[j] ~ normal(0, 5);
		             tau[j] ~ cauchy(0, 2.5);
       }

        sigmab ~ cauchy(0, 2.5);
	      rho ~ uniform(-1, 1);
        nu[2] ~ normal(0, sigmab[2]);

       for (i in 1:Ns){
                 nu[1][i] ~ normal((sigmab[1]/sigmab[2])*rho*nu[2][i], sqrt(sigmabsq[1]*(1 - (rho*rho))));
          for (j in 1:2){
              for (k in 1:Nt)
                        delta[j][i,k] ~ normal(0, tau[j][k]);
        }
    }

    for (n in 1:N){
              TP[n] ~ binomial(Dis[n], p_i[Test[n]][Study[n], 1]);
              TN[n] ~ binomial(NDis[n], p_i[Test[n]][Study[n], 2]);
    }

}
generated quantities{

    vector[2*N] loglik;

    for (n in 1:N)
             loglik[n] = binomial_lpmf(TN[n]| NDis[n], p_i[Test[n]][Study[n], 1]);

    for (n in (N+1):(2*N))
             loglik[n] = binomial_lpmf(TN[n-N]| NDis[n-N], p_i[Test[n-N]][Study[n-N], 2]);

}
'

### Lists of Biomarkers in network meta-analysis

In [ ]:
mk.names=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')

### Data set to used

In [ ]:
filename<-'https://raw.githubusercontent.com/wujinja-cgu/Network-meta-analysis-and-meta-regression-of-diagnostic-tests/main/Sepsis_nma_data.csv'

### Parameters lists

In [ ]:
para.ls<-getpara(filename=filename,names.mk=mk.names)
para<-para.ls$para
data<-para.ls$data

### Prepare data for Network Meta-analysis

In [ ]:
para.as.ls.nma<-list(N=nrow(para),
           Nt=length(unique(para$Test)),
           Ns=length(unique(para$Study)),
           TP=para$TP,
           Dis=para$Dis,
           TN=para$TN,
           NDis=para$NDis,
           Study=as.numeric(para$Study),
           Test=para$Test)

### Compiling model

In [ ]:
stan.model.nma<-stan(model_code=stan.code.nma,
           data=para.as.ls.nma,
           iter=1,
           warmup=0,
           chains=2)

Warning message:
“There were 2 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”


### Running model

In [ ]:
start_time <- Sys.time()
fit.nma<-stan(fit=stan.model.nma,
              data=para.as.ls.nma,
              iter=200,
              chains=4,
              cores=4,
              control=list(adapt_delta=0.99,
                           stepsize=0.01,
                           max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”


Time difference of 17.56225 mins

### Save the results as .RData and .csv

In [ ]:
filename.simdata.save<-'fit_nma_simdata_qSOFAasreference.RData'
filename.results.save<-'out_nma_sepsis3_qSOFAasreference.csv'
results.nma<-fit.extract(fit.data=fit.nma,filename.save=filename.simdata.save)
results.nma.95<-get.nma(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.nma),mk.names=mk.names,
                            conf=95)
write.csv(results.nma.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-analysis

In [ ]:
rownames(results.nma.95$mu)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$mu

,mean_sp,ub_sp,lb_sp,mean_se,ub_se,lb_se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
qSOFA,0.8108337,0.9752205,0.4169209,0.3652054,0.6226168,0.1502642
PCT,0.7457838,0.7787765,0.7209262,0.7321883,0.7591662,0.6975057
presepsin,0.7225757,0.7925544,0.6353709,0.7890475,0.8526443,0.6949692
CRP,0.6922244,0.7412481,0.6023845,0.6768215,0.7158947,0.6233629
CD64,0.7899123,0.9169901,0.5975382,0.7927140,0.9287771,0.5122726
IL-6,0.7453952,0.8327502,0.5845577,0.7072184,0.7842695,0.5855139
sTREM-1,0.6398267,0.9301467,0.2244031,0.5104235,0.7316838,0.1376613
LBP,0.5921977,0.8140960,0.3126832,0.6375726,0.8177326,0.4000812
SIRS,0.5137728,0.6676873,0.3668140,0.7475128,0.8779292,0.5584685


### Print the results of pooled DOR from Network Meta-analysis

In [ ]:
rownames(results.nma.95$dor)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$dor

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,7.199387,30.186290,0.2337816
PCT,8.244554,10.134053,6.8237728
presepsin,10.530337,16.396428,4.8670759
CRP,4.953500,6.627300,3.0820180
CD64,24.370267,80.351082,1.7576558
IL-6,8.125826,13.740402,3.1031175
sTREM-1,4.857427,11.991378,0.2571905
LBP,3.327949,9.770250,0.6645058
SIRS,3.701749,7.920629,1.1141231


### Print the results of pooled Superiority index from Network Meta-analysis

In [ ]:
rownames(results.nma.95$sindex)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,1.0871836,5.000000,0.05263158
PCT,4.0750476,11.000000,0.60000000
presepsin,6.1103810,15.000000,1.00000000
CRP,0.5913189,1.666667,0.11111111
CD64,10.7892525,19.000000,0.33333333
IL-6,3.8141416,13.000000,0.09090909
sTREM-1,0.6099468,5.000000,0.05882353
LBP,0.6484169,7.000000,0.06666667
SIRS,0.7936698,3.000000,0.06666667


### Print the results of pooled Superiority index for Sensitivity from Network Meta-analysis

In [ ]:
rownames(results.nma.95$sindex_se)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_se

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,0.1309599,0.5384615,0.05263158
PCT,2.4387879,5.6666667,0.81818182
presepsin,8.4208813,19.0000000,0.81818182
CRP,0.8634226,1.8571429,0.33333333
CD64,10.6630632,19.0000000,0.33333333
IL-6,1.7876742,5.6666667,0.17647059
sTREM-1,0.6566413,3.0000000,0.05263158
LBP,1.4766411,5.6666667,0.05263158
SIRS,5.8335534,19.0000000,0.33333333


### Print the results of pooled Superiority index for Specificity from Network Meta-analysis

In [ ]:
rownames(results.nma.95$sindex_sp)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_sp

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,10.3761803,19.0000000,0.05263158
PCT,1.7158235,5.6666667,0.53846154
presepsin,1.3156935,5.6666667,0.33333333
CRP,0.6521693,1.2222222,0.17647059
CD64,5.6551506,19.0000000,0.17647059
IL-6,2.2856623,5.6666667,0.17647059
sTREM-1,1.8826688,19.0000000,0.05263158
LBP,0.5058289,3.0000000,0.05263158
SIRS,0.1784606,0.8181818,0.05263158


##################################################################################
###                          Univariable Network meta-regression                         ###
##################################################################################

### Define a function to get the results from Univariable Network Meta-regression

In [ ]:
get.nma.1mk<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	           data.nma<-data.ls$data.nma
	           func.ub<-NULL
	           func.lb<-NULL
	           if(conf==95){
		                func.ub<-get.ub.95
		                func.lb<-get.lb.95
	           } else if(conf==90){
		                     func.ub<-get.ub.90
		                     func.lb<-get.lb.90
	           }
	           data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1se<-data.nma[paste('beta1[1,',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1sp<-data.nma[paste('beta1[2,',1:length(data.ls$mk),']',sep='')]
	           mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	           rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	           or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	           dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                           ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                           lb=array(unlist(lapply(data.nma.dor,func.lb))))
	           sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                             ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                             lb=array(unlist(lapply(data.nma.sindex,func.lb))))
	           sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                              ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                              lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
	           sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                              ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                              lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	           beta1.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.beta1sp,mean))),
	                              ub_sp=array(unlist(lapply(data.nma.beta1sp,func.ub))),
	                              lb_sp=array(unlist(lapply(data.nma.beta1sp,func.lb))),
	                              mean_se=array(unlist(lapply(data.nma.beta1se,mean))),
	                              ub_se=array(unlist(lapply(data.nma.beta1se,func.ub))),
	                              lb_se=array(unlist(lapply(data.nma.beta1se,func.lb))))
	           results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                    mu=mu.info,
	                    rr=rr.info,
	                    or=or.info,
	                    dor=dor.info,
	                    sindex=sindex.info,
	                    sindex_se=sindex_se.info,
	                    sindex_sp=sindex_sp.info,
	                    beta1=beta1.info)
	return(results.nma)
}

### Univariable Network Meta-regression (STAN code)

In [ ]:
stan.code.1mk<-'
data{
   int N;         //number of comparison (n=121)
   int Nt;        //number of test (n=7)
   int Ns;        //number of study (n=107)
   int TP[N];
   int Dis[N];    //diseased
   int TN[N];
   int NDis[N];   //non-diseased
   int Study[N];
   int Test[N];
   int Covar1[N]; //one additional variable
}

parameters{
      matrix[2,Nt] logitmu;
      vector[Ns] nu[2];
      matrix[Ns,Nt] delta[2];
      vector<lower=0>[Nt] tau[2];
      vector<lower=0>[2] sigmab;
      real<lower=-1,upper=1> rho;
      matrix[2,Nt] beta1;
}

transformed parameters{
             matrix[Ns,2] p_i[Nt];
             matrix[2,Nt] MU;
             matrix[2,Nt] RR;
             matrix[2,Nt] OR;
             vector[Nt] DOR;
             vector[Nt] S;
             vector[Nt] S_se;
             vector[Nt] S_sp;
             matrix[Nt,Nt] A;
             matrix[Nt,Nt] B;
             matrix[Nt,Nt] C;
             matrix[Nt, Nt] A_se;
             matrix[Nt, Nt] B_se;
             matrix[Nt, Nt] C_se;
             matrix[Nt, Nt] A_sp;
             matrix[Nt, Nt] B_sp;
             matrix[Nt, Nt] C_sp;

             vector<lower=0>[Nt] tausq[2];
             vector<lower=0>[2] sigmabsq;

             matrix[Nt,Nt] sigmasq[2];
             matrix[Nt,Nt] rhow[2];

for (i in 1:Ns){
  for (j in 1:2){
    for (k in 1:Nt){
               p_i[k][i,j]=inv_logit(logitmu[j,k]+nu[j][i]+delta[j][i,k]+beta1[j,k]*Covar1[i]);
            }
        }
    }

for (j in 1:2){
  for (k in 1:Nt){
            MU[j,k]=mean(col(p_i[k],j));
        }
         tausq[j]=(tau[j]).*(tau[j]);
    }

for (j in 1:2){
  for (k in 1:Nt){
            RR[j,k]=MU[j,k]/MU[j,1];
            OR[j,k]=(MU[j,k]*(1-MU[j,1]))/(MU[j,1]*(1-MU[j,k]));
        }
    }

for (l in 1:Nt){
          DOR[l]=(MU[1,l]*MU[2,l])/((1-MU[1,l])*(1-MU[2,l]));
  for(m in 1:Nt){
            A[l,m]=if_else((MU[1,l]>MU[1,m]) && (MU[2,l]>MU[2,m]),1,0);
            B[l,m]=if_else((MU[1,l]<MU[1,m]) && (MU[2,l]<MU[2,m]),1,0);
            C[l,m]=if_else((MU[1,l]==MU[1,m]) && (MU[2,l]==MU[2,m]),1,0);

            A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
            B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
            C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);

            A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
            B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
            C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
      }
        S[l]=(2*sum(row(A,l))+sum(row(C,l)))/(2*sum(row(B,l))+sum(row(C,l)));
        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
  }

    sigmabsq=(sigmab).*(sigmab);

for (j in 1:2){
  for (k in 1:Nt){
    for (l in 1:Nt){
              sigmasq[j][k,l]=(sigmabsq[j]+tausq[j][k])*((sigmabsq[j]+tausq[j][l]));
              rhow[j][k,l]=sigmabsq[j]/sqrt(sigmasq[j][k,l]);
          }
      }
  }
}

model{
    //----Priors
    for (j in 1:2){
              logitmu[j]~normal(0,5);
              tau[j]~cauchy(0,2.5);

    for (k in 1:Nt){
              beta1[j,k]~normal(0,5);
          }
      }
        sigmab~cauchy(0, 2.5);
        rho~uniform(-1, 1);
        nu[2]~normal(0,sigmab[2]);

    for (i in 1:Ns){
              nu[1][i]~normal((sigmab[1]/sigmab[2])*rho*nu[2][i],sqrt(sigmabsq[1]*(1-(rho*rho))));
      for (j in 1:2){
        for (k in 1:Nt){
                  delta[j][i,k]~normal(0,tau[j][k]);
              }
          }
      }

    for (n in 1:N){
              TP[n]~binomial(Dis[n],p_i[Test[n]][Study[n],1]);
              TN[n]~binomial(NDis[n],p_i[Test[n]][Study[n],2]);
          }
}

    generated quantities{
                vector[2*N] loglik;

    for (n in 1:N){
              loglik[n]=binomial_lpmf(TN[n]|NDis[n],p_i[Test[n]][Study[n],1]);
          }
      for (n in (N+1):(2*N)){
                    loglik[n]=binomial_lpmf(TN[n-N]|NDis[n-N],p_i[Test[n-N]][Study[n-N],2]);
                }
            }
'

### Prepare data for Network Meta-regression

In [ ]:
para.as.ls.1mk<-list(N=nrow(para),
           Nt=length(unique(para$Test)),
           Ns=length(unique(para$Study)),
           TP=para$TP,
           Dis=para$Dis,
           TN=para$TN,
           NDis=para$NDis,
           Study=as.numeric(para$Study),
           Test=para$Test,
           Covar1=data$is.prevalence05)

### Compiling model

In [ ]:
stan.model.1mk<-stan(model_code=stan.code.1mk,
           data=para.as.ls.1mk,
           iter=1,
           warmup=0,
           chains=1)


SAMPLING FOR MODEL 'd63a5a9eefe03ef3da783ca7728595dc' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000815 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 8.15 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: WARNING: No variance estimation is
Chain 1:          performed for num_warmup < 20
Chain 1: 
Chain 1: Iteration: 1 / 1 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 2e-06 seconds (Warm-up)
Chain 1:                0.001921 seconds (Sampling)
Chain 1:                0.001923 seconds (Total)
Chain 1: 


Warning message:
“There were 1 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”


### Running model

In [ ]:
start_time <- Sys.time()
fit.1mk<-stan(fit=stan.model.1mk,
       data=para.as.ls.1mk,
       iter=200,
       chains=4,
       cores=4,
       control=list(adapt_delta=0.99,
              stepsize=0.01,
              max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

Warning message:
“There were 3 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”


Time difference of 21.93722 mins

### Save the results as .RData and .csv

In [ ]:
filename.simdata.save<-'fit_1mk_simdata_sepsis_isprevalence05.RData'
filename.results.save<-'out_1mk_sepsis_isprevalence05.csv'
results.1mk<-fit.extract(fit.data=fit.1mk,filename.save=filename.simdata.save)
results.1mk.95<-get.nma.1mk(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.1mk),mk.names=mk.names,
                            conf=95)
write.csv(results.1mk.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-regression

In [ ]:
rownames(results.1mk.95$mu)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.1mk.95$mu

,mean_sp,ub_sp,lb_sp,mean_se,ub_se,lb_se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
qSOFA,0.8343904,0.9861647,0.5833991,0.3753245,0.6282312,0.13888902
PCT,0.7462204,0.7705313,0.7231538,0.7322352,0.7646050,0.71296368
presepsin,0.7271080,0.7874983,0.6445695,0.8226873,0.8668823,0.76242709
CRP,0.7031980,0.7439010,0.6346436,0.6771432,0.7239936,0.62269597
CD64,0.7924926,0.9247047,0.5879458,0.7151757,0.9395261,0.38651558
IL-6,0.7380014,0.8293991,0.6509445,0.7140065,0.8058153,0.53058898
sTREM-1,0.6157849,0.9636320,0.2331245,0.5978874,0.9358386,0.09638128
LBP,0.5384333,0.9071572,0.1404003,0.4727226,0.9266352,0.08339972
SIRS,0.5311905,0.7471237,0.3060454,0.6831586,0.8875260,0.43340942


### Print the results of pooled DOR from Network Meta-regression

In [ ]:
rownames(results.1mk.95$dor)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.1mk.95$dor

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,9.606980,43.680741,0.31165888
PCT,8.375743,10.141162,6.84635591
presepsin,14.136606,20.110562,7.10854265
CRP,5.111536,6.679264,3.51455177
CD64,17.165532,68.496934,0.93830275
IL-6,7.790352,14.582939,3.55360244
sTREM-1,17.989279,152.796252,0.08999238
LBP,7.756740,35.952940,0.03677282
SIRS,3.914135,14.253367,0.57185014
